## Multiclass Logistic Regression

### One Vs All

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
class LogisticRegression:
        
    def thres(self,n):
        if(n>=0.5):
            return 1
        else:
            return 0
    
    def hyp(self):
        pp= -1 * np.dot(self.X, self.theta)
        prob= (1.0 / (1 + (np.e **(pp))))
        newf=  np.vectorize(self.thres)
        newp= newf(prob)
        return prob
    
        
    def get_cost(self):
        aa= np.dot(np.log(self.hyp().transpose()), self.y)
        bb= np.dot((np.log(1 - self.hyp().transpose())), (1- self.y))
        coss= aa + bb
        mcost = ((-1.0/self.m) * sum(coss))
        return mcost
    
    def fscale(self):
        self.X= self.X / (self.X.max(axis= 0) - self.X.min(axis= 0))
        self.y= self.y / (max(self.y) - min(self.y))
        
    def computeStandard(self):
        from sklearn.linear_model import LogisticRegression as LR
        self.logreg= LR()
        self.logreg.fit(self.X, self.y)

    def preprocess(self, X, y, alpha):
        (self.m, self.n)= X.shape
        self.columns= X.columns
        self.X= X.values
        self.y= y.values
        #self.fscale()
        self.computeStandard()
        self.n+=1
        self.X= np.insert(self.X, 0, 1, axis= 1)
        self.y= self.y[None].transpose()
        self.alpha= alpha
        self.theta= (np.ones(self.n)*0)[None].transpose()
        
          
    def fit(self, X, y, alpha= 0.1):
        self.preprocess(X, y, alpha)
        pc= math.inf
        change= 1
        iteration=0
        print("Change is ",change)
        while(change>0.0000001 and iteration<=350000):
            der = (self.hyp() - self.y).transpose()
            der = np.dot(der , self.X) * self.alpha / self.m
            self.theta= self.theta - der.transpose()

            self.cost= self.get_cost()
            change= pc - self.cost
            pc= self.cost
            iteration += 1
            if(iteration % 1000 == 0):
                print("{}. Cost: {}, theta: {}".format(iteration, self.cost, self.theta.transpose()))
            #print("Change is ",change)
        print('+'*70, "\n{}. Final Cost: {}, theta: {}".format(iteration, self.cost, self.theta.transpose()))
        print("sklearn: theta: ",self.logreg.intercept_, self.logreg.coef_,"\n")
        
            
    def predict(self, X):
        X= X.values
        X= np.insert(X, 0, 1, axis= 1)
        pp= -1 * np.dot(X, self.theta)
        prob= (1.0 / (1 + (np.e **(pp))))
        newf=  np.vectorize(self.thres)
        newp= newf(prob)
        return newp.transpose()[0], prob.transpose()[0]
    
    
    def accuracy(self, y_true, y_pred):
        return sum(y_true == y_pred) / len(y_true)

### Iris Dataset

In [3]:
from sklearn.datasets import load_iris
iris= load_iris()
feature_cols= iris.feature_names
data= pd.DataFrame(iris.data, columns=feature_cols)
data['target']= pd.Series(iris.target)
data= data.sample(frac=1)
data= data.reset_index(drop= True)
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,6.3,2.5,5.0,1.9,2
1,4.6,3.1,1.5,0.2,0
2,4.8,3.0,1.4,0.1,0
3,5.1,3.5,1.4,0.3,0
4,6.4,3.2,4.5,1.5,1


In [4]:
X= data[feature_cols]
y= data['target']
m= data.shape[0]

### Algo

In [5]:
ys= pd.get_dummies(y)
no_cls= ys.shape[1]

In [ ]:
lrs= [None]*no_cls
i =0
for column in ys:
    print("i is {}".format(column))
    lrs[i]= LogisticRegression()
    lrs[i].fit(X, ys[column], alpha= 0.01)
    i+=1

i is 0
Change is  1
1000. Cost: [0.05846262], theta: [[ 0.17511011  0.27046625  0.91548935 -1.47048601 -0.66808307]]
2000. Cost: [0.03138436], theta: [[ 0.2235198   0.35072478  1.17380463 -1.86342883 -0.84581639]]
3000. Cost: [0.02176357], theta: [[ 0.25220136  0.39722705  1.32926112 -2.10075566 -0.95338266]]
4000. Cost: [0.01678327], theta: [[ 0.27267755  0.43010461  1.44132531 -2.27212597 -1.03131427]]
5000. Cost: [0.01372056], theta: [[ 0.28862712  0.45557894  1.52923437 -2.40670904 -1.09272532]]
6000. Cost: [0.01163889], theta: [[ 0.30170134  0.47639476  1.60169581 -2.51773974 -1.1435535 ]]
7000. Cost: [0.01012799], theta: [[ 0.31278519  0.49400664  1.66340558 -2.61236538 -1.18700289]]
8000. Cost: [0.00897918], theta: [[ 0.32240868  0.50927874  1.71719035 -2.69489211 -1.22500369]]
9000. Cost: [0.00807487], theta: [[ 0.33091456  0.52276654  1.76488592 -2.76811842 -1.25881062]]
10000. Cost: [0.00734365], theta: [[ 0.33853734  0.53484834  1.80775309 -2.83396722 -1.28928627]]
11000. Co

55000. Cost: [0.49436938], theta: [[ 3.49452363  0.29537922 -2.314488    1.0604272  -2.56951199]]
56000. Cost: [0.49416772], theta: [[ 3.53795811  0.28737844 -2.31783856  1.06558328 -2.57482999]]
57000. Cost: [0.49397129], theta: [[ 3.58085713  0.27952469 -2.32119289  1.07058586 -2.57992203]]
58000. Cost: [0.49377995], theta: [[ 3.62322813  0.27181556 -2.32455133  1.07544031 -2.58479878]]
59000. Cost: [0.49359355], theta: [[ 3.66507844  0.26424856 -2.32791399  1.08015182 -2.58947032]]
60000. Cost: [0.49341193], theta: [[ 3.70641528  0.25682111 -2.33128076  1.08472535 -2.59394617]]
61000. Cost: [0.49323495], theta: [[ 3.74724574  0.24953059 -2.33465141  1.08916573 -2.5982353 ]]
62000. Cost: [0.49306248], theta: [[ 3.78757679  0.24237434 -2.33802555  1.09347758 -2.60234622]]
63000. Cost: [0.49289439], theta: [[ 3.82741531  0.23534968 -2.34140268  1.09766536 -2.60628696]]
64000. Cost: [0.49273054], theta: [[ 3.86676805  0.22845393 -2.34478222  1.10173336 -2.61006514]]
65000. Cost: [0.4925

53000. Cost: [0.07584858], theta: [[-3.01110316 -3.41626992 -3.37902544  5.02583657  5.65246358]]
54000. Cost: [0.07551809], theta: [[-3.04229609 -3.42939696 -3.39767248  5.04705677  5.68936186]]
55000. Cost: [0.07519722], theta: [[-3.07324606 -3.44222461 -3.41598036  5.06784799  5.72567679]]
56000. Cost: [0.07488551], theta: [[-3.10395977 -3.45476381 -3.43395972  5.08822558  5.76142632]]
57000. Cost: [0.07458253], theta: [[-3.13444357 -3.46702491 -3.45162067  5.10820411  5.79662752]]
58000. Cost: [0.07428788], theta: [[-3.16470356 -3.47901772 -3.4689729   5.12779737  5.83129669]]
59000. Cost: [0.07400119], theta: [[-3.19474555 -3.49075155 -3.48602562  5.14701849  5.86544937]]
60000. Cost: [0.07372211], theta: [[-3.22457509 -3.50223524 -3.50278764  5.16587993  5.8991004 ]]
61000. Cost: [0.07345031], theta: [[-3.2541975  -3.51347719 -3.5192674   5.18439352  5.93226399]]
62000. Cost: [0.07318546], theta: [[-3.28361786 -3.52448538 -3.53547297  5.20257055  5.96495369]]
63000. Cost: [0.0729

### Model Accuracy

In [ ]:
y_p= pd.Series()
for i in range(m):
    XX= X.iloc[i:i+1, :]
    k = 0
    probs= [None] * no_cls
    for column in ys:
        y_pred, probs[k]= lrs[k].predict(XX)
        k+=1
    y_p.set_value(i, probs.index(max(probs)))
print("Model Accuracy is {}".format(sum(y==y_p)/float(len(y))))

### Null Accuracy

In [ ]:
y.value_counts()[0] / len(y)

### Standard Accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
lr= LR()
lr.fit(X, y)
y_pred= lr.predict(X)
from sklearn.metrics import accuracy_score
score= accuracy_score(y, y_pred)
print('Score',score)
print('theta:', lr.coef_)
print('intercept:', lr.intercept_)

## END